In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [2]:
#from google.colab import files
#uploaded = files.upload()

In [3]:
file = open("data for next word prediction.txt", "r", encoding = "utf8")

# store file in list
lines = []
for i in file:
    lines.append(i)

print("$$$$$$$$$$$$$$$$$$$$$$$$$$$")
print(lines)
# Convert list to string
data = ""
for i in lines:
  data = ' '. join(lines) 

#replace unnecessary stuff with space
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')  #new line, carriage return, unicode character --> replace by space

#remove unnecessary spaces 
data = data.split()
data = ' '.join(data)
data[:500]
len(data)

$$$$$$$$$$$$$$$$$$$$$$$$$$$
['The Project Gutenberg eBook of Pride and Prejudice\n', '    \n', 'This ebook is for the use of anyone anywhere in the United States and\n', 'most other parts of the world at no cost and with almost no restrictions\n', 'whatsoever. You may copy it, give it away or re-use it under the terms\n', 'of the Project Gutenberg License included with this ebook or online\n', 'at www.gutenberg.org. If you are not located in the United States,\n', 'you will have to check the laws of the country where you are located\n', 'before using this eBook.\n', '\n', 'Title: Pride and Prejudice\n', '\n', 'Author: Jane Austen\n', '\n', 'Release date: June 1, 1998 [eBook #1342]\n', '                Most recently updated: October 29, 2024\n', '\n', 'Language: English\n', '\n', 'Credits: Chuck Greif and the Online Distributed Proofreading Team at http://www.pgdp.net (This file was produced from images available at The Internet Archive)\n', '\n', '\n', '*** START OF THE PROJECT GUTENBE

220668

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

# saving the tokenizer for predict function
pickle.dump(tokenizer, open('token.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]
len(sequence_data)
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)
sequences = []

for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequences.append(words)
    
print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]
X = []
y = []

for i in sequences:
    X.append(i[0:3])
    y.append(i[3])
    
X = np.array(X)
y = np.array(y)
print("Data: ", X[:10])
print("Response: ", y[:10])
y = to_categorical(y, num_classes=vocab_size)
y[:5]

2401
The Length of sequences are:  11453
Data:  [[  1 425 347]
 [425 347 244]
 [347 244   2]
 [244   2 102]
 [  2 102   3]
 [102   3 183]
 [  3 183  46]
 [183  46 244]
 [ 46 244  11]
 [244  11  22]]
Response:  [244   2 102   3 183  46 244  11  22   1]


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [7]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))
model.summary()

e:\vs code software\python\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [11]:
from tensorflow import keras
from keras.utils import plot_model
#from keras.utils.vis_utils import plot_model

keras.utils.plot_model(model, to_file='plot.png', show_layer_names=True)

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("next_words.h5", monitor='loss', verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))
model.fit(X, y, epochs=70, batch_size=64, callbacks=[checkpoint])

Epoch 1/70
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 461ms/step - loss: 6.8518
Epoch 1: loss improved from inf to 6.62898, saving model to next_words.h5


179/179 ━━━━━━━━━━━━━━━━━━━━ 96s 464ms/step - loss: 6.8506
Epoch 2/70
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 428ms/step - loss: 6.2599
Epoch 2: loss improved from 6.62898 to 6.23779, saving model to next_words.h5


179/179 ━━━━━━━━━━━━━━━━━━━━ 77s 431ms/step - loss: 6.2598
Epoch 3/70
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 450ms/step - loss: 6.0620
Epoch 3: loss improved from 6.23779 to 6.02772, saving model to next_words.h5


179/179 ━━━━━━━━━━━━━━━━━━━━ 81s 453ms/step - loss: 6.0618
Epoch 4/70
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 479ms/step - loss: 5.7075
Epoch 4: loss improved from 6.02772 to 5.72623, saving model to next_words.h5


179/179 ━━━━━━━━━━━━━━━━━━━━ 87s 482ms/step - loss: 5.7076
Epoch 5/70
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 548ms/step - loss: 5.4560
Epoch 5: loss improved from 5.72623 to 5.46922, saving model to next_words.h5


...

179/179 ━━━━━━━━━━━━━━━━━━━━ 144s 536ms/step - loss: 0.2556
Epoch 37/70
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 527ms/step - loss: 0.2799
Epoch 37: loss improved from 0.32712 to 0.30917, saving model to next_words.h5


179/179 ━━━━━━━━━━━━━━━━━━━━ 95s 530ms/step - loss: 0.2800
Epoch 38/70
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 514ms/step - loss: 0.2610
Epoch 38: loss improved from 0.30917 to 0.28118, saving model to next_words.h5


179/179 ━━━━━━━━━━━━━━━━━━━━ 93s 518ms/step - loss: 0.2611
Epoch 39/70
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 506ms/step - loss: 0.2403
Epoch 39: loss improved from 0.28118 to 0.25770, saving model to next_words.h5


179/179 ━━━━━━━━━━━━━━━━━━━━ 141s 510ms/step - loss: 0.2404
Epoch 40/70
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 506ms/step - loss: 0.2021
Epoch 40: loss improved from 0.25770 to 0.23950, saving model to next_words.h5


179/179 ━━━━━━━━━━━━━━━━━━━━ 91s 509ms/step - loss: 0.2024
Epoch 41/70
 24/179 ━━━━━━━━━━━━━━━━━━━━ 1:29 577ms/step - loss: 0.1578

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

# Load the model and tokenizer
model = load_model('next_words.h5')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):

  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array(sequence)
  preds = np.argmax(model.predict(sequence))
  predicted_word = ""
  
  for key, value in tokenizer.word_index.items():
      if value == preds:
          predicted_word = key
          break
  
  print(predicted_word)
  return predicted_word

In [ ]:
while(True):
  text = input("Enter your line: ")
  
  if text == "0":
      print("Execution completed.....")
      break
  
  else:
      try:
          text = text.split(" ")
          text = text[-3:]
          print(text)
        
          Predict_Next_Words(model, tokenizer, text)
          
      except Exception as e:
        print("Error occurred: ",e)
        continue